## Import libraries

Pydantic guarantees the types and constraints of the output model, not the input data.  If you're unsure what this means or how it might affect your usage you should read the section about [Data Conversion](https://docs.pydantic.dev/usage/models/#data-conversion) below. Although validation is not the main purpose of pydantic, you can use this library for custom [validation](https://docs.pydantic.dev/usage/validators/).

In [3]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name = 'Jane Doe'

In [4]:
user = User(id='123')
user_x = User(id='123.45')

ValidationError: 1 validation error for User
id
  value is not a valid integer (type=type_error.integer)

In [5]:
assert user.id == 123
assert user_x.id == 123
assert isinstance(user_x.id, int)  # Note that 123.45 was casted to an int and its value is 123

NameError: name 'user_x' is not defined

In [6]:
assert user.name == 'Jane Doe'

In [7]:
assert user.__fields_set__ == {'id'}

In [9]:
assert user.dict() == dict(user) == {'id': 123, 'name': 'Jane Doe'}

In [10]:
user.id = 321
assert user.id == 321

## Model properties


The example above only shows the tip of the iceberg of what models can do. Models possess the following methods and attributes:

**dict()** : returns a dictionary of the model's fields and values; cf. exporting models


**json()** : returns a JSON string representation dict(); cf. exporting models


**copy()** : returns a copy (by default, shallow copy) of the model; cf. exporting models


**parse_obj()** : a utility for loading any object into a model with error handling if the object is not a dictionary; cf. helper functions


**parse_raw()** : a utility for loading strings of numerous formats; cf. helper functions


**parse_file()** : like parse_raw() but for file paths; cf. helper functions


**from_orm()** : loads data into a model from an arbitrary class; cf. ORM mode


**schema()** : returns a dictionary representing the model as JSON Schema; cf. schema


**schema_json()** : returns a JSON string representation of schema(); cf. schema


**construct()** : a class method for creating models without running validation; cf. Creating models without validation


__fields_set__ : Set of names of fields which were set when the model instance was initialised


__fields__ : a dictionary of the model's fields


__config__ : the configuration class for the model, cf. model config

## Recursive Models

More complex hierarchical data structures can be defined using models themselves as types in annotations.

In [11]:
from pydantic import BaseModel


class Foo(BaseModel):
    count: int
    size: float | None = None


class Bar(BaseModel):
    apple = 'x'
    banana = 'y'


class Spam(BaseModel):
    foo: Foo
    bars: list[Bar]

In [12]:
m = Spam(foo={'count': 4}, bars=[{'apple': 'x1'}, {'apple': 'x2'}])
m

Spam(foo=Foo(count=4, size=None), bars=[Bar(apple='x1', banana='y'), Bar(apple='x2', banana='y')])

In [13]:
m.dict()

{'foo': {'count': 4, 'size': None},
 'bars': [{'apple': 'x1', 'banana': 'y'}, {'apple': 'x2', 'banana': 'y'}]}

## ORM Mode (aka Arbitrary Class Instances)

Pydantic models can be created from arbitrary class instances to support models that map to ORM objects.

To do this:

- The Config property orm_mode must be set to True.
- The special constructor from_orm must be used to create the model instance.

The example here uses SQLAlchemy, but the same approach should work for any ORM.

In [14]:
from sqlalchemy import Column, Integer, String
from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.orm import declarative_base
from pydantic import BaseModel, constr

Base = declarative_base()

### ORM Models

In [15]:
class CompanyOrm(Base):
    __tablename__ = 'companies'
    id = Column(Integer, primary_key=True, nullable=False)
    public_key = Column(String(20), index=True, nullable=False, unique=True)
    name = Column(String(63), unique=True)
    domains = Column(ARRAY(String(255)))

### Pydantic models

In [16]:
class CompanyModel(BaseModel):
    id: int
    public_key: constr(max_length=20)
    name: constr(max_length=63)
    domains: list[constr(max_length=255)]

    class Config:
        orm_mode = True # Important!

### Create a CompanyOrm instance

In [17]:
co_orm = CompanyOrm(
    id=123,
    public_key='foobar',
    name='Testing',
    domains=['example.com', 'foobar.com'],
)
print(co_orm)

### Create a CompanyModel instance from the CompanyOrm instance

In [18]:
co_model = CompanyModel.from_orm(co_orm)
print(co_model)

id=123 public_key='foobar' name='Testing' domains=['example.com', 'foobar.com']


## Reserved names

You may want to name a Column after a reserved SQLAlchemy field. In that case, Field aliases will be convenient:

In [19]:
from pydantic import BaseModel, Field
import sqlalchemy as sa
from sqlalchemy.orm import declarative_base


class MyModel(BaseModel):
    metadata: dict[str, str] = Field(alias='metadata_') # Use 'metadata_' as the alias

    class Config:
        orm_mode = True


Base = declarative_base()


class SQLModel(Base):
    __tablename__ = 'my_table'
    id = sa.Column('id', sa.Integer, primary_key=True)
    # 'metadata' is reserved by SQLAlchemy, hence the '_'
    metadata_ = sa.Column('metadata', sa.JSON)

In [20]:
sql_model = SQLModel(metadata_={'key': 'val'}, id=1)

pydantic_model = MyModel.from_orm(sql_model) # Note that the alias is used here

print(pydantic_model.dict())
#> {'metadata': {'key': 'val'}}
print(pydantic_model.dict(by_alias=True))
#> {'metadata_': {'key': 'val'}}

{'metadata': {'key': 'val'}}
{'metadata_': {'key': 'val'}}


The example above works because aliases have priority over field names for field population. Accessing SQLModel's metadata attribute would lead to a ValidationError.

## Recursive ORM models

ORM instances will be parsed with from_orm recursively as well as at the top level.

Here a vanilla class is used to demonstrate the principle, but any ORM class could be used instead.

In [22]:
from pydantic import BaseModel


class PetCls:
    def __init__(self, *, name: str, species: str):
        self.name = name
        self.species = species


class PersonCls:
    def __init__(self, *, name: str, age: float = None, pets: list[PetCls]):
        self.name = name
        self.age = age
        self.pets = pets


class Pet(BaseModel):
    name: str
    species: str

    class Config:
        orm_mode = True


class Person(BaseModel):
    name: str
    age: float = None
    pets: list[Pet]

    class Config:
        orm_mode = True

In [24]:
bones = PetCls(name='Bones', species='dog')
orion = PetCls(name='Orion', species='cat')
anna = PersonCls(name='Anna', age=20, pets=[bones, orion])
anna_model = Person.from_orm(anna)
print(anna_model)

name='Anna' age=20.0 pets=[Pet(name='Bones', species='dog'), Pet(name='Orion', species='cat')]


## Data binding

Arbitrary classes are processed by pydantic using the GetterDict class (see utils.py), which attempts to provide a dictionary-like interface to any class. You can customise how this works by setting your own sub-class of GetterDict as the value of Config.getter_dict (see config).

You can also customise class validation using root_validators with pre=True. In this case your validator function will be passed a GetterDict instance which you may copy and modify.

The GetterDict instance will be called for each field with a sentinel as a fallback (if no other default value is set). Returning this sentinel means that the field is missing. Any other value will be interpreted as the value of the field.

In [25]:
from pydantic import BaseModel
from typing import Any
from pydantic.utils import GetterDict
from xml.etree.ElementTree import fromstring


xmlstring = """
<User Id="2138">
    <FirstName />
    <LoggedIn Value="true" />
</User>
"""


class UserGetter(GetterDict):

    def get(self, key: str, default: Any) -> Any:

        # element attributes
        if key in {'Id', 'Status'}:
            return self._obj.attrib.get(key, default)

        # element children
        else:
            try:
                return self._obj.find(key).attrib['Value']
            except (AttributeError, KeyError):
                return default


class User(BaseModel):
    Id: int
    Status: str | None
    FirstName: str | None
    LastName: str | None
    LoggedIn: bool

    class Config:
        orm_mode = True
        getter_dict = UserGetter


user = User.from_orm(fromstring(xmlstring))